In [2]:
import mpmath as mp

# ---------- helpers ----------
def _mp_dps_for_lambda(lambda_bits: int) -> int:
    return max(80, int(lambda_bits / 3.3219280948873626) + 60)

def _binom_tail_ge_k(d: int, p, k: int) -> mp.mpf:
    """P[X >= k] for X ~ Bin(d,p) using regularized incomplete beta."""
    if k <= 0:
        return mp.mpf(1)
    if k > d:
        return mp.mpf(0)
    return mp.betainc(k, d - k + 1, 0, p, regularized=True)

# ---------- (1) rigorous UB for P[max > B] via union + exact binomial tail ----------
def bound_union_exact(d: int, m: int, B: int) -> mp.mpf:
    """UB(B) = m * P[Bin(d,1/m) >= B+1]."""
    p = mp.mpf(1) / mp.mpf(m)
    val = mp.mpf(m) * _binom_tail_ge_k(d, p, B + 1)
    return mp.mpf(1) if val > 1 else val

def find_B_union_exact(d: int, m: int, lambda_bits: int):
    """Minimal B s.t. bound_union_exact(d,m,B) <= 2^{-lambda_bits}."""
    mp.mp.dps = _mp_dps_for_lambda(lambda_bits)
    target = mp.power(2, -lambda_bits)
    lo, hi = -1, d
    while hi - lo > 1:
        mid = (lo + hi) // 2
        if bound_union_exact(d, m, mid) <= target:
            hi = mid
        else:
            lo = mid
    return hi, float(bound_union_exact(d, m, hi))

# ---------- Poisson CDF (correct) ----------
def _poisson_cdf(k: int, mu: mp.mpf) -> mp.mpf:
    """
    F(k) = P[Poisson(mu) <= k] = Q(k+1, mu) (regularized upper incomplete gamma)
    """
    if k < 0:
        return mp.mpf(0)
    if mu == 0:
        return mp.mpf(1)
    return mp.gammainc(k + 1, mu, mp.inf, regularized=True)

# ---------- (2) Poisson i.i.d. approx for P[max > B] and E[max] ----------
def approx_prob_max_gt_B_poisson_iid(d: int, m: int, B: int) -> mp.mpf:
    """Approx: P[max > B] ≈ 1 - F(B)^m,  X_i i.i.d. Poi(mu), mu=d/m."""
    mu = mp.mpf(d) / mp.mpf(m)
    F = _poisson_cdf(B, mu)
    if F <= 0:
        return mp.mpf(1)
    if F >= 1:
        return mp.mpf(0)
    return mp.mpf(1) - mp.e**(mp.mpf(m) * mp.log(F))  # 1 - F^m

def find_B_poisson_iid_approx(d: int, m: int, lambda_bits: int):
    """Minimal B s.t. approx_prob_max_gt_B_poisson_iid <= 2^{-lambda_bits} (NOT a bound)."""
    mp.mp.dps = _mp_dps_for_lambda(lambda_bits)
    target = mp.power(2, -lambda_bits)
    lo, hi = -1, d
    while hi - lo > 1:
        mid = (lo + hi) // 2
        if approx_prob_max_gt_B_poisson_iid(d, m, mid) <= target:
            hi = mid
        else:
            lo = mid
    return hi, float(approx_prob_max_gt_B_poisson_iid(d, m, hi))

def approx_E_max_poisson_iid(d: int, m: int, *, tail_eps: float = 1e-30, max_terms: int = 2_000_000):
    """
    Approx E[M] where M=max_i X_i, X_i i.i.d. Poi(mu), mu=d/m.
    Tail-sum: E[M] = sum_{k>=0} (1 - F(k)^m).
    """
    mu = mp.mpf(d) / mp.mpf(m)
    mp.mp.dps = 80
    E = mp.mpf(0)
    small_run = 0
    for k in range(0, max_terms):
        Fk = _poisson_cdf(k, mu)
        if Fk <= 0:
            term = mp.mpf(1)
        elif Fk >= 1:
            term = mp.mpf(0)
        else:
            term = mp.mpf(1) - mp.e**(mp.mpf(m) * mp.log(Fk))
        E += term
        if term < tail_eps:
            small_run += 1
            if small_run >= 50:
                break
        else:
            small_run = 0
    return float(E), {"mu": float(mu), "terms_used": k + 1, "last_term": float(term)}

# ---------- (3) Rigorous upper bound on E[max] via union + exact tail ----------
def upper_E_max_via_union_exact(d: int, m: int, *, tail_eps: float = 1e-30):
    """
    Rigorous: E[M] = sum_{k>=1} P[M >= k]
    and P[M >= k] <= min(1, m * P[Bin(d,1/m) >= k]).
    So:
      E[M] <= sum_{k>=1} min(1, m * P[Bin(d,1/m) >= k]).
    """
    mp.mp.dps = 80
    p = mp.mpf(1) / mp.mpf(m)
    E_ub = mp.mpf(0)
    for k in range(1, d + 1):
        term = mp.mpf(m) * _binom_tail_ge_k(d, p, k)
        if term > 1:
            term = mp.mpf(1)
        E_ub += term
        if term < tail_eps and k > 10:
            break
    return float(E_ub), {"stopped_at_k": k, "last_term": float(term)}

# ---------- convenience wrapper ----------
def compare_B_and_E(d: int, m: int, lambda_bits: int):
    B_rig, ub_at_B = find_B_union_exact(d, m, lambda_bits)
    B_poi, approx_prob = find_B_poisson_iid_approx(d, m, lambda_bits)
    E_poi, meta_poi = approx_E_max_poisson_iid(d, m)
    E_ub, meta_ub = upper_E_max_via_union_exact(d, m)
    return {
        "mu": d / m,
        "B_union_exact_rigorous": {"B": B_rig, "ub_at_B": ub_at_B},
        "B_poisson_iid_approx": {"B": B_poi, "approx_prob_max_gt_B": approx_prob},
        "Emax_poisson_iid_approx": {"E": E_poi, **meta_poi},
        "Emax_union_exact_upper_bound": {"E_ub": E_ub, **meta_ub},
    }


In [3]:
compare_B_and_E(d=1_048_576, m=16384, lambda_bits=40)


{'mu': 64.0,
 'B_union_exact_rigorous': {'B': 141, 'ub_at_B': 5.26075344127133e-13},
 'B_poisson_iid_approx': {'B': 141,
  'approx_prob_max_gt_B': 5.275996424457511e-13},
 'Emax_poisson_iid_approx': {'E': 98.137211438294,
  'mu': 64.0,
  'terms_used': 235,
  'last_term': 1.916357372130041e-56},
 'Emax_union_exact_upper_bound': {'E_ub': 99.07528228791566,
  'stopped_at_k': 186,
  'last_term': 4.598571996060281e-31}}